In [2]:
import pandas as pd 
import geopandas as gpd
import numpy as np

In [3]:
df_election = pd.read_csv('./data/fips_df.csv')
gdf = gpd.read_file('./data/Congressional_Districts.geojson')  # replace with your geojson file path

# Convert all Timestamp objects to strings
gdf = gdf.map(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)

# ...Data Processing...
# Update '00' districts to '01' in GeoJSON data
gdf['DISTRICT'] = gdf['DISTRICT'].replace('00', '01')
df_election['District'] = df_election['District'].apply(lambda x: str(x).zfill(2))

# Convert the STATEFP20 columns to string in both DataFrames
gdf['STATEFP20'] = gdf['STATEFP20'].astype(str).str.zfill(2)
df_election['STATEFP20'] = df_election['STATEFP20'].astype(str).str.zfill(2)

# Convert STATEFP20 columns to string
gdf['STATEFP20'] = gdf['STATEFP20'].astype(str)
df_election['STATEFP20'] = df_election['STATEFP20'].astype(str)
gdf['DISTRICT'] = gdf['DISTRICT'].astype(str)
df_election['District'] = df_election['District'].astype(str)


# Remove any leading or trailing whitespace
gdf['DISTRICT'] = gdf['DISTRICT'].str.strip()
df_election['District'] = df_election['District'].str.strip()

merged_gdf = gdf.merge(df_election, left_on=['STATEFP20', 'DISTRICT'], right_on=['STATEFP20', 'District'], how='left')



In [4]:
merged_gdf.columns

Index(['OBJECTID', 'STATEFP20', 'GEOID20', 'CD118FP', 'NAMELSAD20', 'LSAD20',
       'CDSESSN', 'MTFCC20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20',
       'INTPTLON20', 'OFFICE_ID', 'BIOGUIDE_ID', 'OFFICE_AUDIT_ID', 'PREFIX',
       'FIRSTNAME', 'MIDDLENAME', 'LASTNAME', 'SUFFIX', 'LISTING_NAME',
       'PHONE', 'WEBSITEURL', 'VACANT', 'CONTACTFORMURL', 'PHOTOURL',
       'FACE_BOOK_URL', 'TWITTER_URL', 'YOUTUBE_URL', 'INSTAGRAM_URL',
       'FLICKR_URL', 'PARTY', 'DISTRICT', 'STATE', 'VACANCY', 'ROOM_NUM',
       'HOB', 'COMMITTEE_ASSIGNMENTS', 'LAST_UPDATED', 'SHAPE_Length',
       'SHAPE_Area', 'geometry', 'Unnamed: 0', 'State', 'District', 'Name',
       'Party', 'Incumbent', 'Winner', 'Vote Percentage', 'Raised', 'Spent',
       'Total_Spent_Per_District', 'Percent_Spent'],
      dtype='object')

In [8]:
merged_gdf = merged_gdf[round(merged_gdf['Percent_Spent'], 2)]

,OBJECTID,STATEFP20,GEOID20,CD118FP,NAMELSAD20,LSAD20,CDSESSN,MTFCC20,FUNCSTAT20,ALAND20,...,District,Name,Party,Incumbent,Winner,Vote Percentage,Raised,Spent,Total_Spent_Per_District,Percent_Spent
642,355,46,4600,00,Congressional District (at Large),C1,118,G5200,N,1.963460e+11,...,01,Dusty Johnson,R,True,1.0,81.8,1693504.0,699356.0,699356.0,100.0


In [23]:

# If Winner == 1.0, we use the actual value, else we use None
merged_gdf['display_name'] = np.where(merged_gdf['Winner'] == 1.0, merged_gdf['Name'], None)
merged_gdf['display_party'] = np.where(merged_gdf['Winner'] == 1.0, merged_gdf['Party'], None)
merged_gdf['display_state'] = np.where(merged_gdf['Winner'] == 1.0, merged_gdf['State'], None)
merged_gdf['display_district'] = np.where(merged_gdf['Winner'] == 1.0, merged_gdf['District'], None)
merged_gdf['display_incumbent'] = np.where(merged_gdf['Winner'] == 1.0, merged_gdf['Incumbent'], None)
merged_gdf['display_percent_spent'] = np.where(merged_gdf['Winner'] == 1.0, merged_gdf['Percent_Spent'], None)

In [24]:

merged_gdf.to_parquet('./data/merged_gdf.parquet', compression='gzip')


,OBJECTID,STATEFP20,GEOID20,CD118FP,NAMELSAD20,LSAD20,CDSESSN,MTFCC20,FUNCSTAT20,ALAND20,...,Raised,Spent,Total_Spent_Per_District,Percent_Spent,display_name,display_state,display_party,display_incumbent,display_percent_spent,display_district
0,1,01,0101,01,Congressional District 1,C2,118,G5200,N,1.484369e+10,...,1971321.0,1859349.0,1938322.0,95.925703,Jerry Carl,AL,R,False,95.925703,01
1,1,01,0101,01,Congressional District 1,C2,118,G5200,N,1.484369e+10,...,80095.0,78973.0,1938322.0,4.074297,None,None,None,None,None,None
2,2,01,0102,02,Congressional District 2,C2,118,G5200,N,2.695609e+10,...,650807.0,669368.0,725356.0,92.281307,Barry Moore,AL,R,False,92.281307,02
3,2,01,0102,02,Congressional District 2,C2,118,G5200,N,2.695609e+10,...,56050.0,55988.0,725356.0,7.718693,None,None,None,None,None,None
4,3,01,0103,03,Congressional District 3,C2,118,G5200,N,2.138139e+10,...,1193111.0,1218564.0,1259535.0,96.747133,Mike D Rogers,AL,R,True,96.747133,03
5,3,01,0103,03,Congressional District 3,C2,118,G5200,N,2.138139e+10,...,50273.0,40971.0,1259535.0,3.252867,None,None,None,None,None,None
6,4,01,0104,04,Congressional District 4,C2,118,G5200,N,2.280740e+10,...,1255076.0,1323812.0,1323812.0,100.000000,Robert B Aderholt,AL,R,True,100.0,04
7,5,01,0105,05,Congressional District 5,C2,118,G5200,N,8.697664e+09,...,655365.0,210045.0,210045.0,100.000000,Mo Brooks,AL,R,True,100.0,05
8,6,01,0106,06,Congressional District 6,C2,118,G5200,N,8.332760e+09,...,907219.0,909082.0,909082.0,100.000000,Gary Palmer,AL,R,True,100.0,06
9,7,01,0107,07,Congressional District 7,C2,118,G5200,N,2.815646e+10,...,2168165.0,1495957.0,1495957.0,100.000000,Terri Sewell,AL,D,True,100.0,07
